<a href="https://colab.research.google.com/github/yashh2910/severity/blob/main/vgg16-door.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

In [2]:
! unzip -q '/content/door_balanced_yolo.zip' -d /content/

In [3]:
# Path for train, validation and test datasets
train_path = '/content/door_balanced_yolo/train'
valid_path = '/content/door_balanced_yolo/test'
test_path = '/content/door_balanced_yolo/valid'
IMAGE_SIZE = [224, 224] #Default image size for VGG16
folders = glob('/content/door_balanced_yolo/train/*') #Get number of classes
# ImageDataGenerator can help perform augumentation on existing images. This way, we get more diverse train set.
train_datagen = ImageDataGenerator(rescale = 1./255,brightness_range=(0,1))
validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
#Through flow_from_directory - we create an array of images that can be used for training.
training_set = train_datagen.flow_from_directory(
'/content/door_balanced_yolo/train',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')
validation_set = validation_datagen.flow_from_directory('/content/door_balanced_yolo/valid',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/door_balanced_yolo/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 293 images belonging to 3 classes.
Found 36 images belonging to 3 classes.
Found 33 images belonging to 3 classes.


In [28]:
# Create a VGG16 model, and removing the last layer that is classifying 1000 images. This will be replaced with images classes we have.
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False) #Training with Imagenet weights
# Use this line for VGG19 network. Create a VGG19 model, and removing the last layer that is classifying 1000 images. This will be replaced with images classes we have.
#vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# This sets the base that the layers are not trainable. If we'd want to train the layers with custom data, these two lines can be ommitted.
for layer in vgg.layers:
  layer.trainable = False
x = Flatten()(vgg.output) #Output obtained on vgg16 is now flattened.
prediction = Dense(3, activation='softmax')(x) # We have 5 classes, and so, the prediction is being done on len(folders) - 5 classes
#Creating model object
model = Model(inputs=vgg.input, outputs=vgg.output)
vgg.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
#Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(training_set, validation_data=validation_set, epochs=100, batch_size=32)

Epoch 1/100
5/5 [==============================] - 28s 3s/step - loss: 2.2333 - accuracy: 0.2901 - val_loss: 2.4498 - val_accuracy: 0.3333
Epoch 2/100
5/5 [==============================] - 2s 347ms/step - loss: 1.4933 - accuracy: 0.4164 - val_loss: 1.8708 - val_accuracy: 0.4167
Epoch 3/100
5/5 [==============================] - 2s 365ms/step - loss: 1.0956 - accuracy: 0.4710 - val_loss: 1.4825 - val_accuracy: 0.4167
Epoch 4/100
5/5 [==============================] - 2s 337ms/step - loss: 1.0159 - accuracy: 0.5085 - val_loss: 1.2732 - val_accuracy: 0.4167
Epoch 5/100
5/5 [==============================] - 2s 352ms/step - loss: 0.8463 - accuracy: 0.5563 - val_loss: 1.1404 - val_accuracy: 0.5556
Epoch 6/100
5/5 [==============================] - 2s 341ms/step - loss: 0.7423 - accuracy: 0.6655 - val_loss: 1.0608 - val_accuracy: 0.4722
Epoch 7/100
5/5 [==============================] - 2s 337ms/step - loss: 0.6429 - accuracy: 0.7372 - val_loss: 1.0212 - val_accuracy: 0.5000
Epoch 8/100
5/5

In [6]:
model.save("door.h5")